In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
import torch
from torch import nn
import matplotlib.pyplot as plt
from copy import deepcopy
import pandas as pd
from tqdm import tqdm
from collections import defaultdict
from transformers import AutoTokenizer
import pandas as pd
import seaborn as sns
from types import SimpleNamespace
from datasets import Dataset
from os.path import join as oj
import pickle as pkl
import os
import dvu
dvu.set_style()
import analyze_utils
import sys
sys.path.append('..')
import data
from model_utils import prompt_classification

/Users/chandan/rules/interpretable-autoprompting/.autoprompt/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


Moving 17 files to the new cache system


100%|██████████| 17/17 [00:16<00:00,  1.02it/s]


# Load prompts

In [4]:
# results_acc_dir = oj('/home/chansingh/interpretable-autoprompting', 'results', 'generalization_acc')
results_acc_dir = oj('/Users/chandan/rules/interpretable-autoprompting', 'results', 'generalization_acc')
prompts = pkl.load(open(oj(results_acc_dir, 'prompts_all.pkl'), 'rb'))

**Update a prompt**

In [8]:
prompts.loc['task107_splash_question_to_sql']['autoprompt'] = 'ributed grandfatherExceptionappropri intent Lara'
# warning, this overwrites prompts
# prompts.to_pickle(open(oj(results_acc_dir, 'prompts_all.pkl'), 'wb'))

# Load suffix stuff to merge it in

In [23]:
dir_suffix_search = '/home/chansingh/mntv1/suffix_long_rerun'
r_suffix_search = analyze_utils.load_results_and_cache(dir_suffix_search, save_file='r.pkl', only_keep_scalar=False)
r_suffix_search = pd.read_pickle(os.path.join(dir_suffix_search, 'r.pkl'))
r_suffix_search = analyze_utils.postprocess_results(r_suffix_search,   )
r = r_suffix_search
r.to_pickle('../results/prompt_gen/results_suffix_all.pkl')
r = pd.read_pickle('../results/prompt_gen/results_suffix_all.pkl')

 50%|█████     | 20/40 [00:36<02:08,  6.42s/it]

skipping 6b564f6bdd0dc3f994af2080f044e36558f13743786b13255adf71c4d7ca63fcurch


 68%|██████▊   | 27/40 [00:39<00:19,  1.46s/it]

skipping 885d0a8f6f08d429faf8334b23a057df4b922b421e03740d20e34464f95bbe5csuvv


100%|██████████| 40/40 [00:42<00:00,  1.07s/it]


In [24]:
r0 = r[(r.max_num_tokens == 6) * (r.n_shots == 1) * (r.checkpoint == 'EleutherAI/gpt-j-6B')]
print(r0.shape)
print(list(r0.keys()))

(18, 56)
['task_name', 'n_shots', 'max_dset_size', 'max_digit', 'template_num_init_string', 'template_num_task_phrasing', 'train_split_frac', 'checkpoint', 'max_num_tokens', 'beam_size', 'beam_size_extra', 'use_single_query', 'use_stopwords', 'use_early_stopping', 'use_generic_query', 'float16', 'seed', 'save_dir', 'use_cpu_only', 'use_parallelformers', 'use_cache', 'use_verbose_saving', 'epoch_save_interval', 'batch_size', 'task_name_list', 'suffix_str_init', 'len_suffix_str_init', 'suffix_str_added', 'num_tokens_added', 'num_model_queries', 'running_prob', 'final_answer_pos_initial_token', 'final_answer_full', 'final_answer_added', 'final_model_queries', 'final_num_suffixes_checked', 'final_answer_depth', 'top_prompt', 'top_candidates', 'top_probs', 'final_answer_found', 'Recall @ 3 suffixes', 'Recall @ 5 suffixes', 'Recall @ 10 suffixes', 'Recall @ 15 suffixes', 'Recall @ 20 suffixes', 'Recall @ 25 suffixes', 'Recall @ 30 suffixes', 'Recall @ 40 suffixes', 'Recall @ 50 suffixes', 'R

In [25]:
r0[['task_name', 'top_prompt']]

,task_name,top_prompt
1,double_one,f(x).\n\n
2,max_two,the sum of the input numbers
3,task1147_country_currency,the string representation of the input
5,task1191_food_veg_nonveg,the input.\n\n<|endoftext|>
6,add_two,the sum of the input numbers
7,multiply_two,the sum of the input numbers
10,task1146_country_capital,the name of the capital of
11,subtract_two,-1.\n\n<|endoftext|>
15,square_one,f(x).\n\n
16,divide_two,the result of the division.


The most relevant keys are top_prompt, top_candidates, top_probs.

The relevant keys for metrics are reciprocal_rank, reciprocal_rank_multi.


**Save out top prompts**

In [43]:
idxs = prompts.index
def get_top_prompt(r0, task_name):
    top_prompt = r0[r0['task_name'] == task_name]['top_prompt']
    if len(top_prompt) == 0:
        return np.nan
    else:
        return top_prompt.values[0]
prompts_suff = [get_top_prompt(r0, task_name) for task_name in idxs]
print('WARNING: COPYING VALUE OVER FROM RUN')
prompts_suff[7] = "a function that represents one"
prompts_suff[2] = "a string that records all"

for i, idx, in enumerate(idxs):
    print(i, idxs[i], repr(prompts_suff[i]))
prompts['suffix'] = prompts_suff

0 task088_identify_typo_verification ' the first word of the input'
1 task092_check_prime_classification ' the input.\n\n<|endoftext|>'
2 task107_splash_question_to_sql 'a string that records all'
3 task1146_country_capital ' the name of the capital of'
4 task1147_country_currency ' the string representation of the input'
5 task1149_item_check_edible ' "yes" or "no'
6 task1191_food_veg_nonveg ' the input.\n\n<|endoftext|>'
7 task1336_peixian_equity_evaluation_corpus_gender_classifier 'a function that represents one'
8 task1509_evalution_antonyms ' the first word of the input'
9 task183_rhyme_generation ' the first word of the input'
10 add_two ' the sum of the input numbers'
11 multiply_two ' the sum of the input numbers'
12 divide_two ' the result of the division.'
13 subtract_two ' -1.\n\n<|endoftext|>'
14 max_two ' the sum of the input numbers'
15 first_two ' the sum of the input numbers'
16 square_one ' f(x).\n\n'
17 exp_one ' f(x).\n\n'
18 double_one ' f(x).\n\n'
19 fibonacci_one 

In [41]:
prompts

,manual,autoprompt,iprompt,suffix
task088_identify_typo_verification,The given sentence contains a typo which could...,Sytheanathethethe,Given a string containing the alphabet,the first word of the input
task092_check_prime_classification,"In this task, you need to output 'Yes' if the ...",Abedin AnywaytheONSORED Roses 2050,Yes or no? Is that,the input.\n\n<|endoftext|>
task107_splash_question_to_sql,Generate an SQL statement from a question aski...,nan,Write an SQL to produce output,a string that records all
task1146_country_capital,"In this task, you are given a country name and...",Bringingthethe Stargthethe,How did the economy perform in,the name of the capital of
task1147_country_currency,You are given a country name and you need to r...,renciesthethe Dmitrythe mortg,Select currency code for a new,the string representation of the input
task1149_item_check_edible,Return whether the input item is edible (yes o...,joiningthe Brazil fuckingthethe,Would you like to buy a,"""yes"" or ""no"
task1191_food_veg_nonveg,Return whether the input food dish is vegetari...,Novthethethethethe,yes or no? This is,the input.\n\n<|endoftext|>
task1336_peixian_equity_evaluation_corpus_gender_classifier,Return the gender of the person in the input s...,TurtheFortthetheReviewer,A B C D F#,a function that represents one
task1509_evalution_antonyms,"In this task, you are given an adjective, and ...",prevailingthethe weakestthe wins,What is the opposite of 1,the first word of the input
task183_rhyme_generation,Given an input word generate a word that rhyme...,matesthe dex championshipthethe,bale balbalbab,the first word of the input


In [42]:
# warning, this overwrites prompts
# prompts.to_pickle(open(oj(results_acc_dir, 'prompts_all.pkl'), 'wb'))